# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [6]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
c.execute(""" 
SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit
FROM customers
WHERE (creditLimit > 25000.00) AND (state = 'CA')
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600
3,Julie,Young,6265557265,78934 Hillside Dr.,90700
4,Valarie,Thompson,7605558146,361 Furth Circle,105000
5,Julie,Brown,6505551386,7734 Strong St.,105000
6,Brian,Chandler,2155554369,6047 Douglas Av.,57700
7,Sue,Frick,4085553659,3086 Ingle Ln.,77600
8,Steve,Thompson,3105553722,3675 Furth Circle,55400
9,Sue,Taylor,4155554312,2793 Furth Circle,60300


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [8]:
c.execute("""
SELECT state, AVG(creditLimit)
FROM customers
GROUP BY state;
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [12]:
c.execute("""
SELECT customerName, customerNumber, orderNumber, status, quantityOrdered
FROM customers
LEFT JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [17]:
c.execute("""
SELECT customerName, customerNumber, productName, productCode, SUM(quantityOrdered) AS TotalOrdered
FROM customers LEFT JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING(productCode)
GROUP BY productName, customerName
HAVING TotalOrdered > 10
ORDER BY TotalOrdered DESC;
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,productName,productCode,TotalOrdered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174
...,...,...,...,...,...
2524,Salzburg Collectables,382,1982 Lamborghini Diablo,S24_2972,15
2525,L'ordine Souveniers,386,P-51-D Mustang,S18_2581,15
2526,The Sharp Gifts Warehouse,450,1969 Chevrolet Camaro Z28,S24_3191,13
2527,La Rochelle Gifts,119,1954 Greyhound Scenicruiser,S32_2509,11


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [18]:
c.execute("""
SELECT lastName, firstName, employeeNumber, officeCode
FROM employees
WHERE officeCode IN (SELECT officeCode FROM office
                     JOIN employees
                     USING officeCode
                     GROUP BY)
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,lastName,firstName,employeeNumber,officeCode
0,Murphy,Diane,1002,1
1,Patterson,Mary,1056,1
2,Firrelli,Jeff,1076,1
3,Patterson,William,1088,6
4,Bondur,Gerard,1102,4
5,Bow,Anthony,1143,1
6,Jennings,Leslie,1165,1
7,Thompson,Leslie,1166,1
8,Firrelli,Julie,1188,2
9,Patterson,Steve,1216,2


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!